In [1]:
#https://medium.com/tensorflow/predicting-the-price-of-wine-with-the-keras-functional-api-and-tensorflow-a95d1c2c1b03

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '3'

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [3]:

import itertools
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf



from sklearn.preprocessing import LabelEncoder



from tensorflow import keras

layers = keras.layers



# This code was tested with TensorFlow v1.7

print("You have TensorFlow version", tf.__version__)

/home/anaconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/anaconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/anaconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/anaconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passi

You have TensorFlow version 1.14.0


In [4]:
import pandas as pd
data = pd.read_excel("./train_rev5.xlsx",index_col=0,parse_dates=[0])
df_test = pd.read_excel("./test_rev5.xlsx",index_col=0,parse_dates=[0])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90980 entries, 1 to 129971
Data columns (total 25 columns):
country                   90933 non-null object
description               90980 non-null object
designation               64701 non-null object
price                     84646 non-null float64
province                  90933 non-null object
region_1                  76187 non-null object
region_2                  35436 non-null object
taster_name               72525 non-null object
title                     90980 non-null object
variety                   90980 non-null object
winery                    90980 non-null object
points                    90980 non-null int64
vintage                   87740 non-null float64
vintage_                  90980 non-null int64
taster_name_q2_modify     90980 non-null int64
designation_len_modify    90980 non-null int64
price_modify              90980 non-null int64
province_modify           90980 non-null float64
continent                 90

In [5]:

#!wget -q https://storage.googleapis.com/sara-cloud-ml/wine_data.csv

#data = pd.read_csv("wine_data.csv")

In [5]:
# Get the data: original source is here: https://www.kaggle.com/zynicide/wine-reviews/data
#URL = "https://storage.googleapis.com/sara-cloud-ml/wine_data.csv"
#path = tf.keras.utils.get_file(URL.split('/')[-1], URL)

In [6]:

# Convert the data to a Pandas data frame
#data = pd.read_csv(path)

In [6]:
# Shuffle the data
data = data.sample(frac=1)

# Print the first 5 rows
data.head()

,country,description,designation,price,province,region_1,region_2,taster_name,title,variety,...,designation_len_modify,price_modify,province_modify,continent,grade_germany,grade_italy,grade_spain,grade_france,Rating_Avg,pre_points
id,,,,,,,,,,,,,,,,,,,,,
33535,Spain,A tight note of spearmint defines the nose. Th...,Castell de Raimat,10.0,Catalonia,Costers del Segre,NaN,Michael Schachner,Raimat 2007 Castell de Raimat Cabernet Sauvign...,Cabernet Sauvignon,...,17,1,0.186546,Europe,0,0,0,0,90.0,82
2964,Italy,Fresh and tangy with aromas of white flower an...,NaN,20.0,Northeastern Italy,Colli Orientali del Friuli,NaN,NaN,Guerra Albano 2009 Sauvignon (Colli Orientali ...,Sauvignon,...,0,1,0.301611,Europe,0,0,0,0,93.0,87
50881,US,This wine has the complete butter and baking-s...,Estate Bottled,22.0,California,El Dorado,Sierra Foothills,Jim Gordon,Cedarville 2014 Estate Bottled Viognier (El Do...,Viognier,...,14,1,0.223230,North America,0,0,0,0,93.5,92
85866,Portugal,"From old vines, this is the estate wine of Car...",Carvalhas Vinhas Velhas,60.0,Douro,NaN,NaN,Roger Voss,Quinta das Carvalhas 2012 Carvalhas Vinhas Vel...,Portuguese Red,...,23,1,0.355643,Europe,0,0,0,0,NaN,93
24629,US,"Carneros seems too cool to ripen Zin, and so i...",NaN,25.0,California,Carneros,Napa-Sonoma,NaN,Truchard 2006 Zinfandel (Carneros),Zinfandel,...,0,1,0.223230,North America,0,0,0,0,88.0,85


In [7]:
df_test.head()

,country,description,designation,price,province,region_1,region_2,taster_name,title,variety,...,designation_len_modify,price_modify,province_modify,continent,grade_germany,grade_italy,grade_spain,grade_france,Rating_Avg,pre_points
id,,,,,,,,,,,,,,,,,,,,,
5,France,"A very mineral, flinty character comes through...",Drouhin-Vaudon Les Clos Grand Cru,95.0,Burgundy,Chablis,NaN,Roger Voss,Joseph Drouhin 2013 Drouhin-Vaudon Les Clos Gr...,Chardonnay,...,33,1,0.257697,Europe,0,0,0,3,89.25,95.18257
14,France,"This is a rich wine, just beginning to mature....",NaN,26.0,Provence,Côtes de Provence,NaN,Roger Voss,Château de Berne 2013 Red (Côtes de Provence),Provence red blend,...,0,1,0.278056,Europe,0,0,0,0,NaN,85.64454
15,South Africa,Winemaker: Bruwer Raats. There's a lot to like...,Cape Winemakers Guild Inboet Storie,NaN,Stellenbosch,NaN,NaN,Lauren Buzzeo,B Vintners 2016 Cape Winemakers Guild Inboet S...,Cinsault,...,35,0,0.286437,Africa,0,0,0,0,NaN,88.98295
18,Argentina,A studly Malbec sourced from the Altamira sect...,Catena Nicasia Vineyard,110.0,Mendoza Province,Uco Valley,NaN,Michael Schachner,Bodega Catena Zapata 2004 Catena Nicasia Viney...,Malbec,...,23,2,0.325043,South America,0,0,0,0,NaN,93.57067
21,Italy,"This blend of Cabernet Sauvignon, Merlot and A...",Vigna Alta Il Lagarino di Dionisio,30.0,Southern Italy,Basilicata,NaN,Kerin O’Keefe,Taverna 2010 Vigna Alta Il Lagarino di Dionisi...,Red Blend,...,34,1,0.306304,Europe,0,0,0,0,96.00,84.27711


In [7]:
data['continent'].value_counts()

Europe           42480
North America    38414
South America     5914
Oceania           2650
Africa            1007
Middle East        509
Asia                 6
Name: continent, dtype: int64

In [44]:
# Do some preprocessing to limit the # of wine varities in the dataset
#data = data[pd.notnull(data['country'])]
#data = data[pd.notnull(data['points'])]
data = data.drop(data.columns[0], axis=1) 


In [10]:
data

,country,description,designation,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,points,vintage,continent,grade_germany,grade_italy,grade_spain,grade_france,aged
54653,US,"Sweet and honeyed, this dessert wine has tange...",Patrona,38.0,California,Alexander Valley,Sonoma,NaN,NaN,Stephen & Walker 2012 Patrona Muscat Canelli (...,Muscat Canelli,Stephen & Walker,86,2012,North America,0,0,0,0,0
4745,Italy,"Made entirely from native grape Canaiolo, this...",Soprasassi,60.0,Tuscany,Toscana,NaN,Kerin O’Keefe,@kerinokeefe,I Sodi 2012 Soprasassi Canaiolo (Toscana),Canaiolo,I Sodi,89,2012,Europe,0,0,0,0,0
49645,Italy,"Underbrush, oak and crushed mint aromas lead t...",Saporoia,35.0,Tuscany,Brunello di Montalcino,NaN,Kerin O’Keefe,@kerinokeefe,Baccinetti 2012 Saporoia (Brunello di Montalc...,Sangiovese,Baccinetti,88,2012,Europe,0,0,0,0,0
67149,US,"This is a dark, sappy, saturated wine, and the...",Alice Vineyard,30.0,Washington,Wahluke Slope,Columbia Valley,Paul Gregutt,@paulgwine,Tagaris 2006 Alice Vineyard Malbec (Wahluke Sl...,Malbec,Tagaris,89,2006,North America,0,0,0,0,0
81611,US,"This is a big tannic rustic wine, loaded with ...",Reserve,50.0,Oregon,Applegate Valley,Southern Oregon,Paul Gregutt,@paulgwine,Troon 2014 Reserve Syrah (Applegate Valley),Syrah,Troon,86,2014,North America,0,0,0,0,3
79194,Argentina,Salentein's wines have never been better than ...,Numina Spirit Vineyard Gran Corte,41.0,Mendoza Province,Valle de Uco,NaN,Michael Schachner,@wineschach,Salentein 2014 Numina Spirit Vineyard Gran Cor...,Red Blend,Salentein,93,2014,South America,0,0,0,0,0
78013,Italy,"Aromas of blue flower, dark-skinned fruit, bro...",Vigna La Quercia Riserva,24.0,Tuscany,Chianti Colli Fiorentini,NaN,Kerin O’Keefe,@kerinokeefe,Castelvecchio 2010 Vigna La Quercia Riserva (...,Red Blend,Castelvecchio,86,2010,Europe,0,0,0,0,3
90297,Spain,"This wine's floral, lightly jammy red berry ar...",Peza do Rei,20.0,Galicia,Ribeira Sacra,NaN,Michael Schachner,@wineschach,Adega Cachín 2015 Peza do Rei Mencía (Ribeira ...,Mencía,Adega Cachín,89,2015,Europe,0,0,0,0,0
41425,US,"Meaty and leathery, with sweet, salty bacon no...",Old Vine,36.0,California,St. Helena,Napa,NaN,NaN,Gamble Family Vineyards 2006 Old Vine Syrah (S...,Syrah,Gamble Family Vineyards,93,2006,North America,0,0,0,0,0
37525,Italy,"Aromas of coffee, toast, licorice and a slight...",Serradenari Fossati,75.0,Piedmont,Barolo,NaN,Kerin O’Keefe,@kerinokeefe,Dosio 2012 Serradenari Fossati (Barolo),Nebbiolo,Dosio,90,2012,Europe,0,0,0,0,0


In [46]:

# Split data into train and test
train_size = int(len(data) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 72784
Test size: 18196


In [47]:
print ("size: %d" % len(df_test))

size: 38991


In [48]:

# Train features
description_train = data['description'][:train_size]
variety_train = data['variety'][:train_size]
continent_train = data['continent'][:train_size]

# Train labels
labels_train = data['points'][:train_size]

# Test features
description_test = data['description'][train_size:]
variety_test = data['variety'][train_size:]
continent_test = data['continent'][train_size:]

# Test labels
labels_test = data['points'][train_size:]

In [49]:

# TEST features
description = df_test['description'][:]
variety = df_test['variety'][:]
continent = df_test['continent'][:]



In [50]:

# Create a tokenizer to preprocess our text descriptions
vocab_size = 12000 # This is a hyperparameter, experiment with different values for your dataset
tokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size, char_level=False)
tokenize.fit_on_texts(description_train) # only fit on train

In [51]:
# Wide feature 1: sparse bag of words (bow) vocab_size vector 
description_bow_train = tokenize.texts_to_matrix(description_train)
description_bow_test = tokenize.texts_to_matrix(description_test)
description_bow = tokenize.texts_to_matrix(description)

In [52]:
len(continent_train)

72784

In [53]:
len(continent_test)

18196

In [54]:
# Wide feature 2: one-hot vector of variety categories

# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
#encoder.fit(variety_train)
encoder.fit(continent_train)
continent_train = encoder.transform(continent_train)
continent_test = encoder.transform(continent_test)
continent = encoder.transform(continent)
num_classes = np.max(continent_train) + 1

# Convert labels to one hot
continent_train = keras.utils.to_categorical(continent_train, num_classes)
continent_test = keras.utils.to_categorical(continent_test, num_classes)
continent = keras.utils.to_categorical(continent, num_classes)

In [55]:
# Define our wide model with the functional API
bow_inputs = layers.Input(shape=(vocab_size,))
continent_inputs = layers.Input(shape=(num_classes,))
merged_layer = layers.concatenate([bow_inputs, continent_inputs])
#merged_layer = layers.concatenate([bow_inputs])
merged_layer = layers.Dense(256, activation='relu')(merged_layer)

predictions = layers.Dense(1)(merged_layer)
wide_model = keras.Model(inputs=[bow_inputs, continent_inputs], outputs=predictions)
#wide_model = keras.Model(inputs=[bow_inputs], outputs=predictions)

In [56]:
wide_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
print(wide_model.summary())

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 12000)]      0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 7)]          0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 12007)        0           input_4[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 256)          3074048     concatenate_2[0][0]        

In [57]:
# Deep model feature: word embeddings of wine descriptions
train_embed = tokenize.texts_to_sequences(description_train)
test_embed = tokenize.texts_to_sequences(description_test)
embed = tokenize.texts_to_sequences(description)

max_seq_length = 170
train_embed = keras.preprocessing.sequence.pad_sequences(
    train_embed, maxlen=max_seq_length, padding="post")
test_embed = keras.preprocessing.sequence.pad_sequences(
    test_embed, maxlen=max_seq_length, padding="post")
embed = keras.preprocessing.sequence.pad_sequences(
    embed, maxlen=max_seq_length, padding="post")

In [58]:
# Define our deep model with the Functional API
deep_inputs = layers.Input(shape=(max_seq_length,))
embedding = layers.Embedding(vocab_size, 8, input_length=max_seq_length)(deep_inputs)
embedding = layers.Flatten()(embedding)
embed_out = layers.Dense(1)(embedding)
deep_model = tf.compat.v1.keras.Model(inputs=deep_inputs, outputs=embed_out)
print(deep_model.summary())

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 170)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 170, 8)            96000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1360)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 1361      
Total params: 97,361
Trainable params: 97,361
Non-trainable params: 0
_________________________________________________________________
None


In [59]:
deep_model.compile(loss='mse',
                       optimizer='adam',
                       metrics=['accuracy'])

In [60]:
# Combine wide and deep into one model
merged_out = layers.concatenate([wide_model.output, deep_model.output])
merged_out = layers.Dense(1)(merged_out)
combined_model = tf.compat.v1.keras.Model(wide_model.input + [deep_model.input], merged_out)
print(combined_model.summary())

combined_model.compile(loss='mse',
                       optimizer='adam',
                       metrics=['accuracy'])

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 12000)]      0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 7)]          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 170)]        0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 12007)        0           input_4[0][0]                    
                                                                 input_5[0][0]              

In [61]:

# Run training
combined_model.fit([description_bow_train, continent_train] + [train_embed], labels_train, epochs=10, batch_size=128)

Epoch 1/10
72784/72784 [==============================] - 8s 113us/sample - loss: 623.1785 - acc: 0.0000e+00
Epoch 2/10
72784/72784 [==============================] - 7s 97us/sample - loss: 11.2298 - acc: 0.0000e+00
Epoch 3/10
72784/72784 [==============================] - 7s 95us/sample - loss: 6.4038 - acc: 0.0000e+00
Epoch 4/10
72784/72784 [==============================] - 7s 99us/sample - loss: 4.6370 - acc: 0.0000e+00
Epoch 5/10
72784/72784 [==============================] - 7s 96us/sample - loss: 3.7090 - acc: 0.0000e+00
Epoch 6/10
72784/72784 [==============================] - 7s 98us/sample - loss: 3.1064 - acc: 0.0000e+00
Epoch 7/10
72784/72784 [==============================] - 7s 98us/sample - loss: 2.7228 - acc: 0.0000e+00
Epoch 8/10
72784/72784 [==============================] - 7s 98us/sample - loss: 2.4414 - acc: 0.0000e+00
Epoch 9/10
72784/72784 [==============================] - 8s 106us/sample - loss: 2.1938 - acc: 0.0000e+00
Epoch 10/10
72784/72784 [================

In [62]:
combined_model.evaluate([description_bow_test, continent_test] + [test_embed], labels_test, batch_size=128)

18196/18196 [==============================] - 2s 127us/sample - loss: 3.3851 - acc: 0.0000e+00


[3.3850618614210193, 0.0]

In [63]:
# Generate predictions
predictions = combined_model.predict([description_bow_test, continent_test] + [test_embed])

In [64]:
# Generate predictions
predict_point = combined_model.predict([description_bow, continent] + [embed])

In [32]:
#print("RMSE:", metrics.mean_squared_error(predictions, y_test)**(0.5))

NameError: name 'metrics' is not defined

In [32]:
# Compare predictions with actual values for the first few items in our test dataset
num_predictions = 40
diff = 0

for i in range(num_predictions):
    val = predictions[i]
    print(description_test.iloc[i])
    print('Predicted: ', val[0], 'Actual: ', labels_test.iloc[i], '\n')
    diff += abs(val[0] - labels_test.iloc[i])

Blending 90% Sauvignon Blanc with 10% Sémillon, this white is perfumed in jasmine and apricot honey. On the palate it becomes somewhat fleshier and less expressive, allowing for just a hint of peach.
Predicted:  85.90414 Actual:  87 

Peppery and hot, with blackberry and currant flavors wrapped into firm tannins. Not showing much openness now, but feels like it has the potential to develop with a few years in the bottle.
Predicted:  87.72058 Actual:  86 

Aromas suggesting pressed yellow flower, dried stone fruit and a whiff of toasted nut carry over to the straightforward palate. A hint of almond underscores yellow apple and a citrus note while tangy acidity lends freshness.
Predicted:  86.208824 Actual:  87 

Earthy in aromas, with notes of leafy tomato and red plum. Feels narrow and compact, with herbal, slightly tart flavors of red berry mixed with peppery barrel spice and green herbal tinges. Light, then toasty and spicy on the finish.
Predicted:  86.41095 Actual:  84 

Generous a

In [33]:
# Compare the average difference between actual price and the model's predicted price
print('Average prediction difference: ', diff / num_predictions)

Average prediction difference:  1.22015380859375


In [34]:
df_test

,country,description,designation,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,...,price_1,province_modify,continent,grade_germany,grade_italy,grade_spain,grade_france,aged,Rating_Avg,Maturity_UniqConcat
id,,,,,,,,,,,,,,,,,,,,,
5,France,"A very mineral, flinty character comes through...",Drouhin-Vaudon Les Clos Grand Cru,95.0,Burgundy,Chablis,NaN,Roger Voss,@vossroger,Joseph Drouhin 2013 Drouhin-Vaudon Les Clos Gr...,...,0,0.257697,Europe,0,0,0,3,0,89.25,"E, R"
14,France,"This is a rich wine, just beginning to mature....",NaN,26.0,Provence,Côtes de Provence,NaN,Roger Voss,@vossroger,Château de Berne 2013 Red (Côtes de Provence),...,0,0.278056,Europe,0,0,0,0,0,NaN,NaN
15,South Africa,Winemaker: Bruwer Raats. There's a lot to like...,Cape Winemakers Guild Inboet Storie,NaN,Stellenbosch,NaN,NaN,Lauren Buzzeo,@laurbuzz,B Vintners 2016 Cape Winemakers Guild Inboet S...,...,1,0.286437,Africa,0,0,0,0,0,NaN,NaN
18,Argentina,A studly Malbec sourced from the Altamira sect...,Catena Nicasia Vineyard,110.0,Mendoza Province,Uco Valley,NaN,Michael Schachner,@wineschach,Bodega Catena Zapata 2004 Catena Nicasia Viney...,...,0,0.325043,South America,0,0,0,0,0,NaN,NaN
21,Italy,"This blend of Cabernet Sauvignon, Merlot and A...",Vigna Alta Il Lagarino di Dionisio,30.0,Southern Italy,Basilicata,NaN,Kerin O’Keefe,@kerinokeefe,Taverna 2010 Vigna Alta Il Lagarino di Dionisi...,...,0,0.306304,Europe,0,0,0,0,0,96.00,T
28,US,"Well made in a lighter style, this offers stra...",Hawks View Vineyard,60.0,Oregon,Chehalem Mountains,Willamette Valley,Paul Gregutt,@paulgwine,Torii Mor 2008 Hawks View Vineyard Pinot Noir ...,...,0,0.265387,North America,0,0,0,0,0,94.00,T
30,US,"This is definitely high in alcohol, with a not...",NaN,37.0,California,Napa Valley,Napa,NaN,NaN,Frank Family 2010 Zinfandel (Napa Valley),...,0,0.223230,North America,0,0,0,0,0,91.83,"E, R, T"
43,Austria,"Some ripe honeydew melon, the merest hint of r...",Black Edition,78.0,Niederösterreich,NaN,NaN,Anne Krebiehl MW,@AnneInVino,Ebner-Ebenauer 2012 Black Edition Chardonnay (...,...,0,0.295316,Europe,0,0,0,0,0,NaN,NaN
45,US,"This remains tight and somewhat impenetrable, ...",NaN,32.0,Washington,Walla Walla Valley (WA),Columbia Valley,Paul Gregutt,@paulgwine,Kontos 2010 Merlot (Walla Walla Valley (WA)),...,0,0.285799,North America,0,0,0,0,0,92.00,T


In [35]:
predict_point

array([[94.11557 ],
       [86.350845],
       [91.10566 ],
       ...,
       [82.621574],
       [85.58869 ],
       [86.001274]], dtype=float32)

In [36]:
len(predict_point)

38991

In [37]:
## data['predictions'] = predictions

In [38]:
df_test['points'] = predict_point

In [35]:
df_test.head(5)

,id,country,description,designation,price,province,region_1,region_2,taster_name,taster_twitter_handle,...,variety,winery,vintage,continent,grade_germany,grade_italy,grade_spain,grade_france,aged,points
0,5,France,"A very mineral, flinty character comes through...",Drouhin-Vaudon Les Clos Grand Cru,95.0,Burgundy,Chablis,NaN,Roger Voss,@vossroger,...,Chardonnay,Joseph Drouhin,2013,Europe,0,0,0,3,0,94.727417
1,14,France,"This is a rich wine, just beginning to mature....",NaN,26.0,Provence,Côtes de Provence,NaN,Roger Voss,@vossroger,...,Provence red blend,Château de Berne,2013,Europe,0,0,0,0,0,86.471382
2,15,South Africa,Winemaker: Bruwer Raats. There's a lot to like...,Cape Winemakers Guild Inboet Storie,NaN,Stellenbosch,NaN,NaN,Lauren Buzzeo,@laurbuzz,...,Cinsault,B Vintners,2016,Africa,0,0,0,0,0,89.123093
3,18,Argentina,A studly Malbec sourced from the Altamira sect...,Catena Nicasia Vineyard,110.0,Mendoza Province,Uco Valley,NaN,Michael Schachner,@wineschach,...,Malbec,Bodega Catena Zapata,2004,South America,0,0,0,0,0,91.764969
4,21,Italy,"This blend of Cabernet Sauvignon, Merlot and A...",Vigna Alta Il Lagarino di Dionisio,30.0,Southern Italy,Basilicata,NaN,Kerin O’Keefe,@kerinokeefe,...,Red Blend,Taverna,2010,Europe,0,0,0,0,0,85.896271
5,28,US,"Well made in a lighter style, this offers stra...",Hawks View Vineyard,60.0,Oregon,Chehalem Mountains,Willamette Valley,Paul Gregutt,@paulgwine,...,Pinot Noir,Torii Mor,2008,North America,0,0,0,0,0,86.440361
6,30,US,"This is definitely high in alcohol, with a not...",NaN,37.0,California,Napa Valley,Napa,NaN,NaN,...,Zinfandel,Frank Family,2010,North America,0,0,0,0,0,89.463791
7,43,Austria,"Some ripe honeydew melon, the merest hint of r...",Black Edition,78.0,Niederösterreich,NaN,NaN,Anne Krebiehl MW,@AnneInVino,...,Chardonnay,Ebner-Ebenauer,2012,Europe,0,0,0,0,0,90.172852
8,45,US,"This remains tight and somewhat impenetrable, ...",NaN,32.0,Washington,Walla Walla Valley (WA),Columbia Valley,Paul Gregutt,@paulgwine,...,Merlot,Kontos,2010,North America,0,0,0,0,0,86.691330
9,47,Argentina,"Earthy and gritty cherry, tomato and herbal ar...",NaN,17.0,Other,Río Negro Valley,NaN,Michael Schachner,@wineschach,...,Pinot Noir,Denario,2014,South America,0,0,0,0,0,85.745712


In [39]:
X = df_test[df_test.columns.drop(['country', 'description', 'designation', 'province',
                                'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title',
                                'variety', 'winery',
                                 'price', 'vintage', 'continent', 'grade_germany', 'grade_italy', 'grade_spain', 'grade_france', 'aged'])]

In [41]:
X.to_csv("cho2.csv")

In [153]:
df_test.to_csv("wine_test.csv")

In [40]:
X

,id,points
0,5,94.727417
1,14,86.471382
2,15,89.123093
3,18,91.764969
4,21,85.896271
5,28,86.440361
6,30,89.463791
7,43,90.172852
8,45,86.691330
9,47,85.745712
